In [1]:
import pandas as pd
import numpy as np
import nltk

In [4]:
abc = pd.read_csv('abcnews.csv')
abc.columns

Index(['publish_date', 'headline_text'], dtype='object')

In [5]:
abc_news = abc.sample(100000)

In [6]:
abc_news.shape

(100000, 2)

In [8]:
docs = abc['headline_text'].fillna('').str.lower()
docs = docs.str.replace('[^a-z ]','')

docs_clean = []
stopwords = nltk.corpus.stopwords.words('english')
#stopwords.extend(['','use','one','like','work'])
stemmer = nltk.stem.PorterStemmer()
for doc in docs:
    words = doc.split(' ')
    words_clean = [stemmer.stem(word) for word in words if word not in stopwords]
    words_clean = [word for word in words_clean if word not in stopwords]
    docs_clean.append(words_clean)

In [9]:
docs_clean[0][0:5]

['aba', 'decid', 'commun', 'broadcast', 'licenc']

In [10]:
import gensim
dictionary = gensim.corpora.Dictionary(docs_clean)

In [11]:
docs_bow = []
for doc in docs_clean:
    bow = dictionary.doc2bow(doc)
    docs_bow.append(bow)

In [14]:
lda_model = gensim.models.LdaModel(docs_bow,
                                  id2word = dictionary,
                                  num_topics = 9)

In [16]:
topics = []
for bow in docs_bow:
    doc2topic = pd.DataFrame(lda_model.get_document_topics(docs_bow[0]),
            columns = ['topic_no','prob'])
    topic = doc2topic.sort_values('prob', ascending = False)
    topic = topic.iloc[0]['topic_no']
    topics.append(topic)

In [17]:
lda_model.print_topics()

[(0,
  '0.021*"elect" + 0.017*"adelaid" + 0.015*"australia" + 0.015*"day" + 0.015*"first" + 0.014*"test" + 0.013*"open" + 0.013*"win" + 0.012*"final" + 0.011*"power"'),
 (1,
  '0.023*"melbourn" + 0.014*"say" + 0.013*"hospit" + 0.012*"plan" + 0.012*"labor" + 0.010*"polit" + 0.010*"new" + 0.010*"minist" + 0.010*"nt" + 0.009*"senat"'),
 (2,
  '0.115*"" + 0.026*"year" + 0.019*"world" + 0.012*"nsw" + 0.011*"australian" + 0.010*"coast" + 0.009*"australia" + 0.009*"cup" + 0.009*"deal" + 0.009*"nation"'),
 (3,
  '0.029*"sydney" + 0.023*"fire" + 0.021*"sa" + 0.019*"perth" + 0.018*"polic" + 0.017*"help" + 0.015*"found" + 0.015*"turnbul" + 0.014*"hous" + 0.014*"china"'),
 (4,
  '0.028*"man" + 0.024*"polic" + 0.023*"govern" + 0.023*"charg" + 0.022*"court" + 0.019*"death" + 0.017*"murder" + 0.016*"one" + 0.016*"sex" + 0.015*"canberra"'),
 (5,
  '0.029*"wa" + 0.019*"north" + 0.017*"donald" + 0.016*"die" + 0.016*"home" + 0.014*"brisban" + 0.012*"tasmania" + 0.012*"ban" + 0.010*"law" + 0.010*"take"'),